# Ice thickness inversion

This example shows how to run the OGGM ice thickness inversion model
with various ice parameters: the deformation parameter A and a sliding
parameter (fs).

There is currently no "best" set of parameters for the ice thickness
inversion model. As shown in
[Maussion et al. (2019)](https://www.geosci-model-dev.net/12/909/2019/),
the default parameter set results in global volume estimates which are a bit
larger than previous values. For the global estimate of
[Farinotti et al. (2019)](https://www.nature.com/articles/s41561-019-0300-3),
OGGM participated with a deformation parameter A that is 1.5 times larger than the
generally accepted default value.

There is no reason to think that the ice parameters are the same between
neighboring glaciers. There is currently no "good" way to calibrate them,
or at least no generaly accepted one.
We won't discuss the details here, but we provide a script to illustrate
the sensitivity of the model to this choice.

We also demonstrate how to apply a new global task in OGGM, `workflow.calibrate_inversion_from_consensus` to calibrate the A parameter to match the estimate from [Farinotti et al. (2019)](https://www.nature.com/articles/s41561-019-0300-3).

At the end of this tutorial, we show how to distribute the "flowline thickness" to a glacier map.

## Run

In [ ]:
# Libs
import geopandas as gpd

# Locals
import oggm.cfg as cfg
from oggm import utils, workflow, tasks, graphics

# Initialize OGGM and set up the default run parameters
cfg.initialize(logging_level='WARNING')

# Local working directory (where OGGM will write its output)
WORKING_DIR = utils.gettempdir('OGGM_Inversion')
cfg.PATHS['working_dir'] = WORKING_DIR

# Here we use multiprocessing because we have many glaciers
cfg.PARAMS['use_multiprocessing'] = True

# Default parameters
# Deformation: from Cuffey and Patterson 2010
glen_a = 2.4e-24
# Sliding: from Oerlemans 1997
fs = 5.7e-20

### Read the shapefile 

For the following code to work, you will need to downlad the [hunza_selection](https://github.com/OGGM/training-lahore/raw/main/docs/day_4/hunza_selection.zip) shapefile, extract it on your computer, and upload the files to the classroom directory.

In [ ]:
rgi_df = gpd.read_file('hunza_selection.shp')
rgi_df.plot(edgecolor='k');

This is a selection of 17 glaciers from the Hunza basin:

In [ ]:
rgi_df

In [ ]:
# Go - get the pre-processed glacier directories
# We start at level 3, because we need all data for the inversion
cfg.PARAMS['border'] = 80
base_url = 'https://cluster.klima.uni-bremen.de/~oggm/gdirs/oggm_v1.6/L3-L5_files/2023.1/elev_bands/W5E5'
gdirs = workflow.init_glacier_directories(rgi_df, from_prepro_level=3, prepro_base_url=base_url)

## Inversion 

We estimate the ice thickness for all these glaciers testing various values for the Glen's flow parameter A and the sliding parameter fs. For more information about the inversion procedure in OGGM, visit [the documentation](https://docs.oggm.org/en/stable/inversion.html).

In [ ]:
with utils.DisableLogger(): 
    
    # We test all these multiplicative factors against default A
    factors = [0.1, 0.2, 0.5, 0.8, 1, 1.5, 2, 2.5, 3, 5, 8, 10]

    # Run the inversions tasks with the given factors
    for f in factors:
        print(f'Now computing factor {f}')
        # Without sliding
        suf = '_{:03d}_without_fs'.format(int(f * 10))
        workflow.execute_entity_task(tasks.mass_conservation_inversion, gdirs,
                                     glen_a=glen_a*f, fs=0)
        # Store the results of the inversion only
        utils.compile_glacier_statistics(gdirs, filesuffix=suf,
                                         inversion_only=True)

        # With sliding
        suf = '_{:03d}_with_fs'.format(int(f * 10))
        workflow.execute_entity_task(tasks.mass_conservation_inversion, gdirs,
                                     glen_a=glen_a*f, fs=fs)
        # Store the results of the inversion only
        utils.compile_glacier_statistics(gdirs, filesuffix=suf,
                                         inversion_only=True)
        
print('Done!')

## Results

The data are stored as csv files in the working directory. The easiest way to read them is to use [pandas](http://pandas.pydata.org/)!

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import stats
import os

Let's read the output of the inversion with the default OGGM parameters first:

In [ ]:
df = pd.read_csv(os.path.join(WORKING_DIR, 'glacier_statistics_010_without_fs.csv'), index_col=0)

One way to visualize the output is to plot the volume as a function of area in a log-log plot, illustrating the well known volume-area relationship of mountain glaciers:

In [ ]:
ax = df.plot(kind='scatter', x='rgi_area_km2', y='inv_volume_km3')
ax.semilogx(); ax.semilogy();

As we can see, there is a clear relationship, but it is not perfect. Let's fit a line to these data (the "volume-area scaling law"):

In [ ]:
# Fit in log space 
dfl = np.log(df[['inv_volume_km3', 'rgi_area_km2']])
slope, intercept, r_value, p_value, std_err = stats.linregress(dfl.rgi_area_km2.values, dfl.inv_volume_km3.values)

In their paper, [Bahr et al. (1997)](https://agupubs.onlinelibrary.wiley.com/doi/10.1029/97JB01696) describe this relationship as:

$$V = \alpha S^{\gamma}$$

With V the volume in km$^3$, S the area in km$^2$ and $\alpha$ and $\gamma$ the scaling parameters (0.034 and 1.375, respectively). How does OGGM compare to these?

In [ ]:
print('power: {:.3f}'.format(slope))
print('slope: {:.3f}'.format(np.exp(intercept)))

In [ ]:
xlim = [df['rgi_area_km2'].min(), df['rgi_area_km2'].max()]
ax = df.plot(kind='scatter', x='rgi_area_km2', y='inv_volume_km3', label='OGGM glaciers')
ax.plot(xlim, np.exp(intercept) * (xlim ** slope), color='C3', label='Fitted line')
ax.semilogx(); ax.semilogy();
ax.legend();

## Sensitivity analysis 

Now, let's read the output files of each run separately, and compute the regional volume out of them:

In [ ]:
dftot = pd.DataFrame(index=factors)
for f in factors:
    # Without sliding
    suf = '_{:03d}_without_fs'.format(int(f * 10))
    fpath = os.path.join(WORKING_DIR, 'glacier_statistics{}.csv'.format(suf))
    _df = pd.read_csv(fpath, index_col=0, low_memory=False)
    dftot.loc[f, 'without_sliding'] = _df.inv_volume_km3.sum()
    
    # With sliding
    suf = '_{:03d}_with_fs'.format(int(f * 10))
    fpath = os.path.join(WORKING_DIR, 'glacier_statistics{}.csv'.format(suf))
    _df = pd.read_csv(fpath, index_col=0, low_memory=False)
    dftot.loc[f, 'with_sliding'] = _df.inv_volume_km3.sum()

And plot them:

In [ ]:
dftot.plot();
plt.xlabel('Factor of Glen A (default 1)'); plt.ylabel('Regional volume (km$^3$)');

As you can see, there is quite a difference between the solutions. In particular, close to the default value for Glen A, the regional estimates are very sensitive to small changes in A. The calibration of A is a problem that has yet to be resolved by global glacier models...

## Calibrate to match an existing volume estimate 

Here, one "best Glen A" is found in order that the total inverted volume of the glaciers of gdirs fits to the estimate from Farinotti et al. (2019).

In [ ]:
# when we use all glaciers, no Glen A could be found within the range [0.1,10] that would match the consensus estimate
# usually, this is applied on larger regions where this error should not occur ! 
cdf = workflow.calibrate_inversion_from_consensus(gdirs, filter_inversion_output=False, apply_fs_on_mismatch=True)

In [ ]:
cdf.sum()

Note that here we calibrate the Glen A parameter to a value that is equal for all glaciers of gdirs, i.e. we calibrate to match the total volume of all glaciers and not to match them individually.

*just as a side note, "vol_bsl_itmix_m3" means volume below sea level and is therefore zero for these Alpine glaciers!*

## Distributed ice thickness 

The OGGM inversion and dynamical models use the "1D" flowline assumption: for some applications, you might want to use OGGM to create distributed ice thickness maps. Currently, OGGM implements two ways to "distribute" the flowline thicknesses, but only [the simplest one](https://docs.oggm.org/en/stable/generated/oggm.tasks.distribute_thickness_per_altitude.html) works robustly:

In [ ]:
gdir = gdirs[12]

In [ ]:
# Distribute
workflow.execute_entity_task(tasks.distribute_thickness_per_altitude, [gdir]);

In [ ]:
graphics.plot_distributed_thickness(gdir, figsize=(12, 8))